In [16]:
import sys
from pathlib import Path

In [17]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

In [18]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel

In [19]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.asc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    print(compiled_query)
    print("")
    print(str(query))
    # results = session.exec(query).all()
    # print(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at ASC
 LIMIT :param_1


In [20]:
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)

    query = (
        select(
            bucket,
            EventModel.page,
            func.count()
        )
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page
        )
        .order_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    results = session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 6, 6, 0, 0, tzinfo=datetime.timezone.utc), '/about', 2467),
 (datetime.datetime(2025, 6, 6, 0, 0, tzinfo=datetime.timezone.utc), '/contact', 2470),
 (datetime.datetime(2025, 6, 6, 0, 0, tzinfo=datetime.timezone.utc), '/pages', 2544),
 (datetime.datetime(2025, 6, 6, 0, 0, tzinfo=datetime.timezone.utc), '/pricing', 2519)]
